In [1]:
import openpathsampling as paths

In [2]:
from openpathsampling.experimental.storage.storage import GeneralStorage
from openpathsampling.experimental.storage.class_info import ClassInfoContainer, ClassInfo
from openpathsampling.experimental.storage.storage import universal_schema
from openpathsampling.experimental.storage.sql_backend import SQLStorageBackend
import openpathsampling.experimental.storage.serialization_helpers as serialization
from openpathsampling.experimental.storage import tools

import openpathsampling.engines.toy as toys

from openpathsampling.experimental.storage.ops_storage import OPSStorage, OPSClassInfoContainer

import numpy as np

In [3]:
serialize_sim = lambda obj: {'uuid': serialization.get_uuid(obj), 'json': serialization.to_json_obj(obj)}
deserialize_sim = serialization.from_json_obj

In [4]:
from openpathsampling.netcdfplus import StorableObject
default_class_info = ClassInfo(table='simulation_objects',
                               cls=StorableObject,
                               serializer=serialize_sim,
                               deserializer=deserialize_sim)

### Save to storage

This has already been checked in previous notebooks

In [5]:
schema = {
    'simulation_objects': [('json', 'json_obj'), ('class_idx', 'int')]
}

In [6]:
backend = SQLStorageBackend("test.sql", mode='w')
storage = OPSStorage.from_backend(
    backend=backend,
    schema=schema,
    class_info=OPSClassInfoContainer(default_info=default_class_info)
)

In [7]:
pes = (
    toys.OuterWalls([1.0, 1.0], [0.0, 0.0]) +
    toys.Gaussian(-0.7, [12.0, 12.0], [0.0, 0.4]) +
    toys.Gaussian(-0.7, [12.0, 12.0], [-0.5, -0.5]) +
    toys.Gaussian(-0.7, [12.0, 12.0], [0.5, -0.5])
)

topology=toys.Topology(
    n_spatial = 2,
    masses =[1.0, 1.0],
    pes = pes
)
integ = toys.LangevinBAOABIntegrator(dt=0.02, temperature=0.1, gamma=2.5)

options={
    'integ' : integ,
    'n_frames_max' : 5000,
    'n_steps_per_frame' : 1
}

engine = toys.Engine(
    options=options,
    topology=topology
)

template = toys.Snapshot(
    coordinates=np.array([[-0.5, -0.5]]), 
    velocities=np.array([[0.0,0.0]]),
    engine=engine
)

In [8]:
template_uuid = str(template.__uuid__)

In [9]:
import logging
import sys

root = logging.getLogger()
root.setLevel(logging.DEBUG)

ch = logging.StreamHandler(sys.stdout)
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
root.addHandler(ch)

In [10]:
storage.save(template)
storage.save(engine)

2020-05-19 10:04:50,153 - openpathsampling.experimental.storage.storage - DEBUG - Starting save
2020-05-19 10:04:50,157 - openpathsampling.experimental.storage.sql_backend - DEBUG - Looking for 1 UUIDs
2020-05-19 10:04:50,160 - openpathsampling.experimental.storage.sql_backend - DEBUG - New block of 1 UUIDs
2020-05-19 10:04:50,167 - openpathsampling.experimental.storage.sql_backend - DEBUG - Found 0 UUIDs
2020-05-19 10:04:50,171 - openpathsampling.experimental.storage.storage - DEBUG - Listing all objects to save
2020-05-19 10:04:50,250 - openpathsampling.experimental.storage.storage - DEBUG - Checking if objects already exist in database
2020-05-19 10:04:50,257 - openpathsampling.experimental.storage.sql_backend - DEBUG - Looking for 11 UUIDs
2020-05-19 10:04:50,258 - openpathsampling.experimental.storage.sql_backend - DEBUG - New block of 11 UUIDs
2020-05-19 10:04:50,261 - openpathsampling.experimental.storage.sql_backend - DEBUG - Found 0 UUIDs
2020-05-19 10:04:50,263 - openpathsamp

In [11]:
eng_uuid = engine.__uuid__

In [12]:
list(storage.backend.metadata.tables.keys())

['schema', 'metadata', 'uuid', 'tables', 'simulation_objects', 'snapshot0']

In [13]:
storage.close()

In [14]:
# TODO: should make sure here that we error if we try to get anything out of storage

### Reload from storage

In [15]:
backend = SQLStorageBackend("test.sql", mode='r')
storage = OPSStorage.from_backend(
    backend=backend,
    schema=backend.schema,
    class_info=OPSClassInfoContainer(default_info=default_class_info)
)

2020-05-19 10:04:50,428 - openpathsampling.experimental.storage.storage - INFO - Missing info from 1 dynamically-registered tables
2020-05-19 10:04:50,429 - openpathsampling.experimental.storage.ops_storage - INFO - Attempting to register missing table snapshot0 (<class 'openpathsampling.engines.toy.snapshot.ToySnapshot'>)
2020-05-19 10:04:50,438 - openpathsampling.experimental.storage.ops_storage - INFO - Found 1 possible lookups
2020-05-19 10:04:50,439 - openpathsampling.experimental.storage.ops_storage - INFO - Lookups for tables: dict_keys(['snapshot0'])
2020-05-19 10:04:50,441 - openpathsampling.experimental.storage.storage - INFO - Successfully registered 1 missing tables
2020-05-19 10:04:50,445 - openpathsampling.experimental.storage.storage - DEBUG - Starting to load 1 objects
2020-05-19 10:04:50,448 - openpathsampling.experimental.storage.storage - DEBUG - Getting internal structure of 1 non-cached objects
2020-05-19 10:04:50,449 - openpathsampling.experimental.storage.sql_bac

2020-05-19 10:04:50,612 - openpathsampling.experimental.storage.storage - DEBUG - Loading 0 objects; creating 0 lazy proxies
2020-05-19 10:04:50,613 - openpathsampling.experimental.storage.storage - DEBUG - Identifying classes for 0 lazy proxies
2020-05-19 10:04:50,614 - openpathsampling.experimental.storage.sql_backend - DEBUG - Looking for 0 UUIDs
2020-05-19 10:04:50,615 - openpathsampling.experimental.storage.sql_backend - DEBUG - New block of 0 UUIDs
2020-05-19 10:04:50,618 - openpathsampling.experimental.storage.sql_backend - DEBUG - Found 0 UUIDs
2020-05-19 10:04:50,619 - openpathsampling.experimental.storage.storage - DEBUG - Reconstructing from 0 objects
2020-05-19 10:04:50,620 - openpathsampling.experimental.storage.storage - DEBUG - Starting to load 1 objects
2020-05-19 10:04:50,621 - openpathsampling.experimental.storage.storage - DEBUG - Getting internal structure of 0 non-cached objects
2020-05-19 10:04:50,622 - openpathsampling.experimental.storage.storage - DEBUG - Loadi

getting `class_info` when we need to reload

**NOTE:** I think most of this can come from the first instance; serialization/deserialization is the default, so we just need to get the type -- but we *don't* need to have the fully registered `class_info` ... class type can be another entry in the `tables` table

can we get the need for a class

In [16]:
backend.number_to_table

{0: 'simulation_objects', 1: 'snapshot0'}

In [17]:
storage.class_info['simulation_objects']

ClassInfo(table=simulation_objects, cls=<class 'openpathsampling.netcdfplus.base.StorableObject'>, lookup_result=<class 'openpathsampling.netcdfplus.base.StorableObject'>, find_uuids=None)

In [18]:
storage.load([str(eng_uuid)])

2020-05-19 10:04:50,681 - openpathsampling.experimental.storage.storage - DEBUG - Starting to load 1 objects
2020-05-19 10:04:50,684 - openpathsampling.experimental.storage.storage - DEBUG - Getting internal structure of 0 non-cached objects
2020-05-19 10:04:50,685 - openpathsampling.experimental.storage.storage - DEBUG - Loading 0 objects; creating 0 lazy proxies
2020-05-19 10:04:50,686 - openpathsampling.experimental.storage.storage - DEBUG - Identifying classes for 0 lazy proxies
2020-05-19 10:04:50,687 - openpathsampling.experimental.storage.sql_backend - DEBUG - Looking for 0 UUIDs
2020-05-19 10:04:50,688 - openpathsampling.experimental.storage.sql_backend - DEBUG - New block of 0 UUIDs
2020-05-19 10:04:50,690 - openpathsampling.experimental.storage.sql_backend - DEBUG - Found 0 UUIDs
2020-05-19 10:04:50,692 - openpathsampling.experimental.storage.storage - DEBUG - Reconstructing from 0 objects


In [19]:
engine = storage.load([str(eng_uuid)])[0]

2020-05-19 10:04:50,702 - openpathsampling.experimental.storage.storage - DEBUG - Starting to load 1 objects
2020-05-19 10:04:50,704 - openpathsampling.experimental.storage.storage - DEBUG - Getting internal structure of 0 non-cached objects
2020-05-19 10:04:50,705 - openpathsampling.experimental.storage.storage - DEBUG - Loading 0 objects; creating 0 lazy proxies
2020-05-19 10:04:50,707 - openpathsampling.experimental.storage.storage - DEBUG - Identifying classes for 0 lazy proxies
2020-05-19 10:04:50,709 - openpathsampling.experimental.storage.sql_backend - DEBUG - Looking for 0 UUIDs
2020-05-19 10:04:50,711 - openpathsampling.experimental.storage.sql_backend - DEBUG - New block of 0 UUIDs
2020-05-19 10:04:50,723 - openpathsampling.experimental.storage.sql_backend - DEBUG - Found 0 UUIDs
2020-05-19 10:04:50,725 - openpathsampling.experimental.storage.storage - DEBUG - Reconstructing from 0 objects


In [20]:
engine.to_dict()

{'options': {'n_frames_max': 5000,
  'on_max_length': 'fail',
  'on_nan': 'fail',
  'retries_when_nan': 2,
  'retries_when_error': 0,
  'retries_when_max_length': 0,
  'on_retry': 'full',
  'on_error': 'fail',
  'integ': <openpathsampling.engines.toy.integrators.LangevinBAOABIntegrator at 0x1137dfa20>,
  'n_steps_per_frame': 1},
 'topology': <openpathsampling.engines.toy.topology.ToyTopology at 0x1137dfb38>}

In [21]:
storage.load([template_uuid])

2020-05-19 10:04:50,740 - openpathsampling.experimental.storage.storage - DEBUG - Starting to load 1 objects
2020-05-19 10:04:50,743 - openpathsampling.experimental.storage.storage - DEBUG - Getting internal structure of 1 non-cached objects
2020-05-19 10:04:50,749 - openpathsampling.experimental.storage.sql_backend - DEBUG - Looking for 1 UUIDs
2020-05-19 10:04:50,750 - openpathsampling.experimental.storage.sql_backend - DEBUG - New block of 1 UUIDs
2020-05-19 10:04:50,753 - openpathsampling.experimental.storage.sql_backend - DEBUG - Found 1 UUIDs
2020-05-19 10:04:50,756 - openpathsampling.experimental.storage.storage - DEBUG - Loading 1 objects; creating 0 lazy proxies
2020-05-19 10:04:50,758 - openpathsampling.experimental.storage.storage - DEBUG - Identifying classes for 0 lazy proxies
2020-05-19 10:04:50,760 - openpathsampling.experimental.storage.sql_backend - DEBUG - Looking for 0 UUIDs
2020-05-19 10:04:50,762 - openpathsampling.experimental.storage.sql_backend - DEBUG - New blo

In [22]:
snapshot = storage.load([template_uuid])[0]

2020-05-19 10:04:50,780 - openpathsampling.experimental.storage.storage - DEBUG - Starting to load 1 objects
2020-05-19 10:04:50,782 - openpathsampling.experimental.storage.storage - DEBUG - Getting internal structure of 0 non-cached objects
2020-05-19 10:04:50,783 - openpathsampling.experimental.storage.storage - DEBUG - Loading 0 objects; creating 0 lazy proxies
2020-05-19 10:04:50,784 - openpathsampling.experimental.storage.storage - DEBUG - Identifying classes for 0 lazy proxies
2020-05-19 10:04:50,785 - openpathsampling.experimental.storage.sql_backend - DEBUG - Looking for 0 UUIDs
2020-05-19 10:04:50,786 - openpathsampling.experimental.storage.sql_backend - DEBUG - New block of 0 UUIDs
2020-05-19 10:04:50,788 - openpathsampling.experimental.storage.sql_backend - DEBUG - Found 0 UUIDs
2020-05-19 10:04:50,789 - openpathsampling.experimental.storage.storage - DEBUG - Reconstructing from 0 objects


In [23]:
snapshot.__uuid__

139948483702057181973983775567520989208

In [24]:
template_uuid

'139948483702057181973983775567520989208'

In [25]:
engine.__uuid__

139948483702057181973983775567520989206

In [26]:
snap0 = storage.backend.metadata.tables['snapshot0']

In [27]:
snap0

Table('snapshot0', MetaData(bind=Engine(sqlite:///test.sql)), Column('idx', INTEGER(), table=<snapshot0>, primary_key=True, nullable=False), Column('uuid', VARCHAR(), table=<snapshot0>), Column('velocities', BLOB(), table=<snapshot0>), Column('coordinates', BLOB(), table=<snapshot0>), Column('engine', VARCHAR(), table=<snapshot0>), schema=None)

In [28]:
with backend.engine.connect() as conn:
    results = list(conn.execute(snap0.select()))
results

[(1, '139948483702057181973983775567520989208', b'\x00\x00\x00\x00\x00\x00\x00\x00', b'\x00\x00\x00\xbf\x00\x00\x00\xbf', '139948483702057181973983775567520989206')]

In [29]:
results[0]

(1, '139948483702057181973983775567520989208', b'\x00\x00\x00\x00\x00\x00\x00\x00', b'\x00\x00\x00\xbf\x00\x00\x00\xbf', '139948483702057181973983775567520989206')

In [30]:
storage.class_info[template]

ClassInfo(table=snapshot0, cls=<class 'openpathsampling.engines.toy.snapshot.ToySnapshot'>, lookup_result=('139948483702057181973983775567520989206', <class 'openpathsampling.engines.toy.snapshot.ToySnapshot'>), find_uuids=<openpathsampling.experimental.storage.serialization_helpers.SchemaFindUUIDs object at 0x1137bbef0>)

In [31]:
storage.class_info[engine]

ClassInfo(table=simulation_objects, cls=<class 'openpathsampling.netcdfplus.base.StorableObject'>, lookup_result=<class 'openpathsampling.netcdfplus.base.StorableObject'>, find_uuids=None)

In [32]:
storage.class_info

SerializationSchema(default_info=ClassInfo(table=simulation_objects, cls=<class 'openpathsampling.netcdfplus.base.StorableObject'>, lookup_result=<class 'openpathsampling.netcdfplus.base.StorableObject'>, find_uuids=None), class_info_list=[ClassInfo(table=simulation_objects, cls=<class 'openpathsampling.netcdfplus.base.StorableObject'>, lookup_result=<class 'openpathsampling.netcdfplus.base.StorableObject'>, find_uuids=None), ClassInfo(table=snapshot0, cls=<class 'openpathsampling.engines.toy.snapshot.ToySnapshot'>, lookup_result=('139948483702057181973983775567520989206', <class 'openpathsampling.engines.toy.snapshot.ToySnapshot'>), find_uuids=<openpathsampling.experimental.storage.serialization_helpers.SchemaFindUUIDs object at 0x1137bbef0>)])